# CREATION BDD POSTGRES

In [1]:
import os # Gestion mot de passe dans l'environnement

In [2]:
import psycopg2 # A installer dans l'environnement

In [3]:
import utils

## FONCTIONS DE CONNEXION A POSTGRES

In [4]:
# 5 arguments pour se connecter, localhost, 5432 
# http://www.python-simple.com/python-autres-modules-non-standards/psycopg2.php
def ouvrir_connection(nom_bdd, utilisateur, mot_passe, host='localhost', port=5432):
    try:
        conn = psycopg2.connect(dbname=nom_bdd, user=utilisateur, password=mot_passe, host=host, port=5432)
    # sqlite3.Error  --> psycopg2.Error
    except psycopg2.Error as e:
        print("Erreur lors de la connection à la base de données")
        print(e)
        return None
    # autocommit non dispo sur Sql3
    conn.set_session(autocommit=True)
    return conn

In [5]:
def creer_table(conn, pgsql_creation_table):
    try:
        cursor = conn.cursor()
        cursor.execute(pgsql_creation_table)
        conn.commit()
    except psycopg2.Error as e:
        print("Erreur lors de la création de la table")
        print(e)
        return
    cursor.close()
    print("La table a été créée avec succès")

def creer_colonne(conn, pgsql_creation_colonne):
    try:
        cursor = conn.cursor()
        cursor.execute(pgsql_creation_colonne)
        conn.commit()
    except psycopg2.Error as e:
        print("Erreur lors de la création de la colonne")
        print(e)
        return
    cursor.close()
    print("La colonne a été créée avec succès")
    
def inserer_donnees(conn, pgsql_insertion_table, donnees):
    try:
        cursor = conn.cursor()
        for d in donnees:
            cursor.execute(pgsql_insertion_table, d)
        conn.commit()
    except psycopg2.Error as e:
        print("Erreur lors de l'insertion des données")
        print(e)
        return
    cursor.close()
    print("Les données ont été insérées avec succès")

def lire_donnees(conn, pgsql_requete):
    try:
        cursor = conn.cursor()
        cursor.execute(pgsql_requete)
        conn.commit()
    except psycopg2.Error as e:
        print("Erreur lors de la lecture des données")
        print(e)
        return None
    
    print("Les données ont été lues avec succès")
    data = []
    for row in cursor:
        data.append(row)    
   
    cursor.close()
    return data

def lire_donnees(conn, pgsql_requete):
    try:
        cursor = conn.cursor()
        cursor.execute(pgsql_requete)
        conn.commit()
    except psycopg2.Error as e:
        print("Erreur lors de la lecture des données")
        print(e)
        return None
    
    print("Les données ont été lues avec succès")
    data = []
    for row in cursor:
        data.append(row)    
   
    cursor.close()
    return data

def lire_requete_sql(conn, pgsql_requete):
    try:
        cursor = conn.cursor()
        cursor.execute(pgsql_requete)
        conn.commit()
    except psycopg2.Error as e:
        print("Erreur lors de la lecture des données")
        print(e)
        return None
    
    print("Les données ont été lues avec succès")
    data = []
    for row in cursor:
        data.append(row)
        print("{} | Nombre d'avis : {} | Moyenne : {}".format(*row))
    
    cursor.close()
    

## CREATION DES TABLES ET INSERTION DES DONNEES DES FICHIERS CSV Movie / Links / Tags / Ratings

In [6]:
ma_base_donnees = "Movie 2"
utilisateur = "postgres"
mot_passe = os.environ.get('pg_psw') # Prend le mot de passe crée en local bash admin

In [7]:
conn = ouvrir_connection(ma_base_donnees, utilisateur, mot_passe)

type(conn)

psycopg2.extensions.connection

In [8]:
from pathlib import Path
# Répertoire dataset (small)
repertoire_data = Path('data/')
# Chemin vers les différents fichiers de données
chemin_movies = repertoire_data / 'movies.csv'
chemin_links = repertoire_data / 'links.csv'
chemin_ratings = repertoire_data / 'ratings.csv'
chemin_tags = repertoire_data / 'tags.csv'

In [9]:
movies_dict = utils.lire_csv_dict(chemin_movies)

In [10]:
links_dict = utils.lire_csv_dict(chemin_links)

In [11]:
ratings_dict = utils.lire_csv_dict(chemin_ratings)

In [12]:
tags_dict = utils.lire_csv_dict(chemin_tags)

### REQUETES SQL CREATION TABLES ET INSERTIONS DONNEES

In [13]:
# SERIAL PRIMARY key https://www.tutorialspoint.com/postgresql/postgresql_using_autoincrement.htm
pgsql_creer_table = """
    CREATE TABLE IF NOT EXISTS movies (
    movieId INT UNIQUE PRIMARY KEY,
    title VARCHAR NOT NULL,
    genres VARCHAR NOT NULL);
    
    CREATE TABLE IF NOT EXISTS links (
    movieId INT PRIMARY KEY NOT NULL,
    imdbId VARCHAR,
    tmdbId VARCHAR,
    FOREIGN KEY (movieId) REFERENCES movies(movieId));
    
    CREATE TABLE IF NOT EXISTS ratings (
    userId INT NOT NULL,
    movieId INT NOT NULL,
    rating REAL NOT NULL,
    timestamp INT NOT NULL,
    PRIMARY KEY (userId, movieId),
    foreign key(movieId) REFERENCES movies(movieId));
    
    CREATE TABLE IF NOT EXISTS tags (
    userId INT NOT NULL,
    movieId INT NOT NULL,
    tag VARCHAR NOT NULL,
    timestamp INT NOT NULL,
    FOREIGN KEY (movieId) REFERENCES movies(movieId));
"""

pgsql_creer_colonne = """
    ALTER TABLE movies
    ADD years integer;
    
    ALTER TABLE movies
    ADD titre VARCHAR;
"""

pgsql_inserer_movies = """
    INSERT INTO movies 
    (movieId, title, genres)
    VALUES (%(movieId)s, %(title)s, %(genres)s);
"""

pgsql_inserer_links = """
    INSERT INTO links 
    (movieId, imdbId, tmdbId)
    VALUES (%(movieId)s, %(imdbId)s, %(tmdbId)s);
"""

pgsql_inserer_ratings = """
    INSERT INTO ratings 
    (userId, movieId, rating, timestamp)
    VALUES (%(userId)s, %(movieId)s, %(rating)s, %(timestamp)s);
"""
pgsql_inserer_tags = """
    INSERT INTO tags 
    (userId, movieId, tag, timestamp)
    VALUES (%(userId)s, %(movieId)s, %(tag)s, %(timestamp)s);
"""

#### CREATION TABLES

In [14]:
creer_table(conn, pgsql_creer_table)

La table a été créée avec succès


#### INSERTION DONNEES MOVIES

In [15]:
inserer_donnees(conn, pgsql_inserer_movies, movies_dict)

Les données ont été insérées avec succès


#### INSERTION DONNEES LINKS

In [16]:
inserer_donnees(conn, pgsql_inserer_links, links_dict)

Les données ont été insérées avec succès


#### INSERTION DONNEES RATINGS

In [17]:
inserer_donnees(conn, pgsql_inserer_ratings, ratings_dict)

Les données ont été insérées avec succès


#### INSERTION DONNEES TAGS

In [18]:
inserer_donnees(conn, pgsql_inserer_tags, tags_dict)

Les données ont été insérées avec succès


#### CREATION COLONNES

In [19]:
creer_colonne(conn, pgsql_creer_colonne)

La colonne a été créée avec succès


In [20]:
conn.close()

### Fin

## LECTURE ET ANALYSE DES DONNEES

In [21]:
conn = ouvrir_connection(ma_base_donnees, utilisateur, mot_passe)

#### LECTURE 5 PREMIERS ENREGISTRMENTS DE LA TABLE MOVIES - CHECK INSERTION DONNEES

In [22]:
Lecture_Data = """
    SELECT *
    FROM movies
    LIMIT 5;
"""

In [23]:
Donnees = lire_donnees(conn, Lecture_Data)

Les données ont été lues avec succès


In [24]:
Donnees

[(1,
  'Toy Story (1995)',
  'Adventure|Animation|Children|Comedy|Fantasy',
  None,
  None),
 (2, 'Jumanji (1995)', 'Adventure|Children|Fantasy', None, None),
 (3, 'Grumpier Old Men (1995)', 'Comedy|Romance', None, None),
 (4, 'Waiting to Exhale (1995)', 'Comedy|Drama|Romance', None, None),
 (5, 'Father of the Bride Part II (1995)', 'Comedy', None, None)]

In [25]:
conn.close()

## COMPTER LE NOMBRE D'ENREGISTREMENTS EFFECTUES

In [26]:
conn.close()

In [27]:
conn = ouvrir_connection(ma_base_donnees, utilisateur, mot_passe)

In [28]:
pgsql_comptage = """
    SELECT COUNT(*)
    FROM movies;
"""

In [29]:
nb_movies = lire_donnees(conn, pgsql_comptage)

Les données ont été lues avec succès


In [30]:
nb_movies

[(9742,)]

In [31]:
conn.close()

## LES 15 FILMS LES MIEUX NOTES

In [32]:
conn = ouvrir_connection(ma_base_donnees, utilisateur, mot_passe)

In [33]:
Sql_Top_15 = """
    SELECT title, count(rating), CAST(AVG(rating) AS DECIMAL (19,2)) \
    FROM movies INNER JOIN ratings on movies.movieId=ratings.movieId \
    GROUP BY title \
    ORDER BY avg(rating) DESC \
    LIMIT 15
"""

In [34]:
Top_15 = lire_requete_sql(conn, Sql_Top_15)

Les données ont été lues avec succès
Mystery of the Third Planet, The (Tayna tretey planety) (1981) | Nombre d'avis : 1 | Moyenne : 5.00
Garden of Words, The (Koto no ha no niwa) (2013) | Nombre d'avis : 1 | Moyenne : 5.00
Brother (Brat) (1997) | Nombre d'avis : 1 | Moyenne : 5.00
Into the Woods (1991) | Nombre d'avis : 1 | Moyenne : 5.00
Dream of Light (a.k.a. Quince Tree Sun, The) (Sol del membrillo, El) (1992) | Nombre d'avis : 1 | Moyenne : 5.00
Meantime (1984) | Nombre d'avis : 1 | Moyenne : 5.00
Nasu: Summer in Andalusia (2003) | Nombre d'avis : 1 | Moyenne : 5.00
Holy Motors (2012) | Nombre d'avis : 1 | Moyenne : 5.00
Watching the Detectives (2007) | Nombre d'avis : 1 | Moyenne : 5.00
Eva (2011) | Nombre d'avis : 1 | Moyenne : 5.00
Story of Women (Affaire de femmes, Une) (1988) | Nombre d'avis : 1 | Moyenne : 5.00
Connections (1978) | Nombre d'avis : 1 | Moyenne : 5.00
Go for Zucker! (Alles auf Zucker!) (2004) | Nombre d'avis : 1 | Moyenne : 5.00
Girls About Town (1931) | Nombre

## LES 15 FILMS LES MIEUX NOTES AYANT PLUS DE 40 NOTES RECUES

In [35]:
Sql_Top_15_Rating40 = """
    SELECT title, COUNT(rating), AVG(rating) \
    FROM movies INNER JOIN ratings on movies.movieId=ratings.movieId \
    GROUP BY title \
    HAVING COUNT(rating) > 40 \
    ORDER BY AVG(rating) DESC \
    LIMIT 15
"""

In [36]:
Top_15_Rating40 = lire_requete_sql(conn, Sql_Top_15_Rating40)

Les données ont été lues avec succès
Shawshank Redemption, The (1994) | Nombre d'avis : 317 | Moyenne : 4.429022082018927
Lawrence of Arabia (1962) | Nombre d'avis : 45 | Moyenne : 4.3
Godfather, The (1972) | Nombre d'avis : 192 | Moyenne : 4.2890625
Fight Club (1999) | Nombre d'avis : 218 | Moyenne : 4.272935779816514
Cool Hand Luke (1967) | Nombre d'avis : 57 | Moyenne : 4.271929824561403
Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964) | Nombre d'avis : 97 | Moyenne : 4.268041237113402
Rear Window (1954) | Nombre d'avis : 84 | Moyenne : 4.261904761904762
Godfather: Part II, The (1974) | Nombre d'avis : 129 | Moyenne : 4.25968992248062
Departed, The (2006) | Nombre d'avis : 107 | Moyenne : 4.252336448598131
Goodfellas (1990) | Nombre d'avis : 126 | Moyenne : 4.25
Casablanca (1942) | Nombre d'avis : 100 | Moyenne : 4.24
Dark Knight, The (2008) | Nombre d'avis : 149 | Moyenne : 4.238255033557047
Usual Suspects, The (1995) | Nombre d'avis : 204 | Moyenne : 4.2

## LES UTILISATEURS LES PLUS ACTIFS ET LA MOYENNE DE LEURS NOTES ATTRIBUEES

In [37]:
Sql_Top_Users = """
    SELECT UserId, COUNT(rating), AVG(rating) \
    FROM movies INNER JOIN ratings on movies.movieId=ratings.movieId \
    GROUP BY UserId \
    ORDER BY COUNT(rating) DESC \
    LIMIT 15
"""

In [38]:
Top_Users = lire_requete_sql(conn, Sql_Top_Users)

Les données ont été lues avec succès
414 | Nombre d'avis : 2698 | Moyenne : 3.391957005189029
599 | Nombre d'avis : 2478 | Moyenne : 2.6420500403551253
474 | Nombre d'avis : 2108 | Moyenne : 3.398956356736243
448 | Nombre d'avis : 1864 | Moyenne : 2.8473712446351933
274 | Nombre d'avis : 1346 | Moyenne : 3.235884101040119
610 | Nombre d'avis : 1302 | Moyenne : 3.6885560675883258
68 | Nombre d'avis : 1260 | Moyenne : 3.233730158730159
380 | Nombre d'avis : 1218 | Moyenne : 3.6732348111658455
606 | Nombre d'avis : 1115 | Moyenne : 3.6573991031390136
288 | Nombre d'avis : 1055 | Moyenne : 3.1459715639810426
249 | Nombre d'avis : 1046 | Moyenne : 3.6964627151051626
387 | Nombre d'avis : 1027 | Moyenne : 3.2585199610516065
182 | Nombre d'avis : 977 | Moyenne : 3.5112589559877176
307 | Nombre d'avis : 975 | Moyenne : 2.6656410256410257
603 | Nombre d'avis : 943 | Moyenne : 3.5079533404029695


## TOP 10 DES FILMS RECEVANT LE PLUS DE TAGS

In [39]:
Sql_Top_Tags = """
    SELECT title, COUNT(tag) as nb_tags
    FROM tags INNER JOIN movies on tags.movieId=movies.movieId
    GROUP BY title
    ORDER BY nb_tags DESC
    LIMIT 10
"""

In [40]:
Top_Tags = lire_donnees(conn, Sql_Top_Tags)

Les données ont été lues avec succès


In [41]:
Top_Tags

[('Pulp Fiction (1994)', 181),
 ('Fight Club (1999)', 54),
 ('2001: A Space Odyssey (1968)', 41),
 ('Léon: The Professional (a.k.a. The Professional) (Léon) (1994)', 35),
 ('Eternal Sunshine of the Spotless Mind (2004)', 34),
 ('Big Lebowski, The (1998)', 32),
 ('Donnie Darko (2001)', 29),
 ('Inception (2010)', 26),
 ('Star Wars: Episode IV - A New Hope (1977)', 26),
 ('Suicide Squad (2016)', 19)]

## FILMS AYANT EU LE PLUS DE TAG IDENTIQUE

In [42]:
Sql_Top_Tags2 = """
    SELECT title, COUNT(tag) as nb_tags, tag
    FROM tags INNER JOIN movies on tags.movieId=movies.movieId
    GROUP BY tag, title
    ORDER BY nb_tags DESC
    LIMIT 10
"""

In [43]:
Top_Tags2 = lire_donnees(conn, Sql_Top_Tags2)

Les données ont été lues avec succès


In [44]:
Top_Tags2

[('Inception (2010)', 3, 'thought-provoking'),
 ('Donnie Darko (2001)', 3, 'hallucinatory'),
 ('Twelve Monkeys (a.k.a. 12 Monkeys) (1995)', 3, 'time travel'),
 ('Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964)',
  3,
  'dark comedy'),
 ('Fight Club (1999)', 3, 'dark comedy'),
 ('Donnie Darko (2001)', 3, 'dreamlike'),
 ('Star Wars: Episode IV - A New Hope (1977)', 3, 'sci-fi'),
 ('Star Wars: Episode IV - A New Hope (1977)', 3, 'classic sci-fi'),
 ('Step Brothers (2008)', 3, 'will ferrell'),
 ('Memento (2000)', 3, 'twist ending')]

## TOP 25 TAGS LES PLUS UTILISES

In [45]:
Sql_Top_Tags3 = """
    SELECT tag, COUNT(tag) AS nb_tags
    FROM tags
    GROUP BY tag
    ORDER BY COUNT(tag) DESC
    LIMIT 25
"""

In [46]:
Top_Tags3 = lire_donnees(conn, Sql_Top_Tags3)

Les données ont été lues avec succès


In [47]:
Top_Tags3

[('In Netflix queue', 131),
 ('atmospheric', 36),
 ('thought-provoking', 24),
 ('superhero', 24),
 ('funny', 23),
 ('surreal', 23),
 ('Disney', 23),
 ('religion', 22),
 ('psychology', 21),
 ('dark comedy', 21),
 ('quirky', 21),
 ('sci-fi', 21),
 ('suspense', 20),
 ('twist ending', 19),
 ('crime', 19),
 ('visually appealing', 19),
 ('politics', 18),
 ('mental illness', 16),
 ('music', 16),
 ('time travel', 16),
 ('comedy', 15),
 ('aliens', 15),
 ('dark', 15),
 ('mindfuck', 14),
 ('dreamlike', 14)]

## CREATION COLONNE YEARS

In [48]:
Sql_Colonne_Years = """
    ALTER TABLE movies
    ADD years integer
"""

In [49]:
creer_colonne(conn, Sql_Colonne_Years)

Erreur lors de la création de la colonne
ERREUR:  la colonne « years » de la relation « movies » existe déjà



In [50]:
Colonne_Years

NameError: name 'Colonne_Years' is not defined